# 1. SETTINGS

In [24]:
# import packages
import pandas as pd
import numpy as np
import os

In [25]:
# pandas options
pd.set_option('display.max_columns', None)

In [26]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [27]:
# garbage collection
import gc
gc.enable()

In [28]:
# helper functions
import functions
from functions import *

# 2. IMPORT

In [29]:
# import data
train = pd.read_csv('../data/X_train.csv', sep = ',', decimal = '.', error_bad_lines = False)
y     = pd.read_csv('../data/y_train.csv', sep = ',', decimal = '.', error_bad_lines = False)
test  = pd.read_csv('../data/X_test.csv',  sep = ',', decimal = '.', error_bad_lines = False)
print(train.shape)
print(y.shape)
print(test.shape)

(8880, 31)
(8880, 2)
(2960, 31)


b'Skipping line 2168: expected 31 fields, saw 33\nSkipping line 4822: expected 31 fields, saw 37\nSkipping line 4859: expected 31 fields, saw 37\nSkipping line 7342: expected 31 fields, saw 37\n'


In [30]:
# check data
train.head()

,id,images_count,image_width,image_height,image_url,product_description,product_size,material,age,warranty,year,color,product_width,wifi,condition,product_length,shoe_size,vintage,brand,author,editor,product_height,weight,price,category,sub_category_1,sub_category_2,sub_category_3,sub_category_4,product_name,store_name
0,0,3,3458.0,2552.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Superbe petit top bustier avec explosion de co...,44,100 % polyester,NaN,NaN,NaN,Multicolore,NaN,NaN,bon état,NaN,NaN,False,NaN,NaN,NaN,NaN,200.0,4.5,mode,"tops, t-shirts, débardeurs femme",NaN,NaN,NaN,Top bustier multicolore,Emmaüs 88 Neufchateau
1,1,2,2486.0,2254.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,"Radio ITT Océnic Flirt, année 70\nPour déco",NaN,Plastique,NaN,NaN,NaN,Jaune,NaN,NaN,en l'état,NaN,NaN,True,ITT Océanic,NaN,NaN,NaN,1000.0,15.0,mobilier - deco,bibelots et objets déco,NaN,NaN,NaN,Radio ITT Océanic,Communauté Emmaüs Thouars (magasin Parthenay)
2,2,3,1536.0,1536.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Veste boléro à manches courtes NÛMPH. Gris chi...,40,"Polyester, coton, laine",NaN,NaN,NaN,Gris,NaN,NaN,neuf,NaN,NaN,False,Nûmph,NaN,NaN,NaN,360.0,16.0,label selection,mode,mode femme,NaN,NaN,NaN,Label Emmaüs Chambéry
3,3,2,1100.0,1100.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Encrier plumier en verre. De forme carrée le ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bon état,NaN,NaN,NaN,NaN,NaN,NaN,NaN,520.0,24.0,mobilier - deco,autres objets,NaN,NaN,NaN,Encrier Plumier en verre,Emmaüs Lyon
4,4,2,450.0,450.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Si vous souhaitez acquérir un appareil puissan...,NaN,NaN,NaN,6 mois,NaN,Blanc,NaN,True,reconditionné,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,139.0,multimédia,smartphones,NaN,NaN,NaN,HUAWEI Honor 5X Blanc - Débloqué - Très bon état.,La Bootique des Ateliers du Bocage


In [31]:
# check data
test.head()

,id,images_count,image_width,image_height,image_url,product_description,product_size,material,age,warranty,year,color,product_width,wifi,condition,product_length,shoe_size,vintage,brand,author,editor,product_height,weight,price,category,sub_category_1,sub_category_2,sub_category_3,sub_category_4,product_name,store_name
0,0,1,616.0,616.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,<p><strong>Livre d'occasion écrit par Stephen ...,NaN,NaN,NaN,NaN,1999.0,NaN,NaN,NaN,bon état,NaN,NaN,NaN,NaN,Stephen King,J'Ai Lu,NaN,149.0,1.8,librairie,NaN,NaN,NaN,NaN,Peur bleue - Stephen King,Booki - la Librairie Solidaire de Retrilog
1,1,2,3000.0,4000.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Eléguant haut transparent pour femme imprimé ...,XXL,polyester,NaN,NaN,NaN,Multicolore,NaN,NaN,bon état,NaN,NaN,False,Woman,NaN,NaN,NaN,100.0,3.0,mode,"tops, t-shirts, débardeurs femme",NaN,NaN,NaN,vêtement femme Woman,Emmaüs 88 Neufchateau
2,2,3,2448.0,2448.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Collants bleu motifs en losange discret femme ...,NaN,"85 % polyamide, 15% élasthanne.",NaN,NaN,NaN,Bleu,NaN,NaN,neuf,NaN,NaN,False,KUNERT,NaN,NaN,NaN,200.0,3.0,mode,accessoires femme,label selection,NaN,NaN,Collants bleu motifs en losange discret femme ...,Emmaüs Sacré Dressing !
3,3,10,2448.0,2448.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Un beau vase à couvercle en porcelaine\nde la ...,NaN,Porcelaine,NaN,NaN,NaN,Multicolore,NaN,NaN,bon état,NaN,NaN,NaN,Kaiser Porcelaine,NaN,NaN,NaN,2130.0,20.0,mobilier - deco,Vases et Pots,NaN,NaN,NaN,Vase à Couvercle,Emmaüs Vannes - la Boutique en ligne
4,4,3,2753.0,2974.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Ancien Boulier d'écolier en bois . Les enfants...,NaN,NaN,NaN,NaN,NaN,Multicolore,NaN,NaN,en l'état,NaN,NaN,True,NaN,NaN,NaN,NaN,340.0,7.5,enfance,"jouets anciens, jouets en bois",NaN,NaN,NaN,Ancien Boulier d'écolier en bois,Emmaüs 88 Neufchateau


# 3. MERGER

In [32]:
# target variable
train['duration'] = y['duration']
test['duration']  = np.nan

In [33]:
# align columns
train = train.reindex_axis(sorted(train.columns), axis = 1)
test  = test.reindex_axis(sorted(test.columns),   axis = 1)

# check equalty
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [34]:
# concatenate
df = pd.concat([train, test], axis = 0)
df = df.reset_index(drop = True)
del train, test
print(df.shape)

(11840, 32)


In [35]:
# check distributions
df.describe()

,duration,id,image_height,image_width,images_count,price,product_height,product_length,product_width,shoe_size,weight,year
count,8880.000000,11840.00000,11770.000000,11770.000000,11840.000000,11840.000000,163.000000,131.000000,153.00000,300.000000,1.183600e+04,1.983000e+03
mean,0.991667,3699.50000,1800.660918,1804.888785,3.654645,21.795489,63.036810,67.487023,45.79281,39.033333,8.965613e+03,1.166698e+04
std,0.820628,2598.90433,1103.081066,1021.465572,2.057843,45.683665,31.945389,41.744412,28.43733,2.920156,9.095875e+04,4.311602e+05
min,0.000000,0.00000,64.000000,58.000000,0.000000,0.500000,9.000000,23.000000,10.00000,16.000000,1.000000e-01,0.000000e+00
25%,0.000000,1479.75000,962.750000,1000.000000,2.000000,6.000000,45.750000,35.000000,30.00000,37.000000,2.520000e+02,1.979000e+03
50%,1.000000,2959.50000,1536.000000,1536.000000,3.000000,10.000000,60.000000,50.000000,40.00000,39.000000,5.000000e+02,1.998000e+03
75%,2.000000,5919.25000,2448.000000,2448.000000,5.000000,22.000000,72.000000,80.000000,50.00000,41.000000,1.000000e+03,2.007000e+03
max,2.000000,8879.00000,6016.000000,5472.000000,29.000000,2200.000000,198.000000,250.000000,180.00000,48.000000,2.700000e+06,1.920193e+07


# 4. PROCESSING

### MISSING VALUES

In [36]:
# check missings
count_missings(df)

,Total,Percent
sub_category_4,11748,99.222973
wifi,11739,99.146959
warranty,11713,98.927365
product_length,11709,98.893581
product_width,11687,98.707770
product_height,11677,98.623311
age,11672,98.581081
shoe_size,11540,97.466216
sub_category_3,10612,89.628378
author,9957,84.096284


In [37]:
# fill imputed cases with 0
nas = ['image_url']
for var in nas:
    df[var].fillna(0, inplace = True)

### VARIABLE TYPES

In [38]:
# check data types
df.dtypes

age                     object
author                  object
brand                   object
category                object
color                   object
condition               object
duration               float64
editor                  object
id                       int64
image_height           float64
image_url               object
image_width            float64
images_count             int64
material                object
price                  float64
product_description     object
product_height         float64
product_length         float64
product_name            object
product_size            object
product_width          float64
shoe_size              float64
store_name              object
sub_category_1          object
sub_category_2          object
sub_category_3          object
sub_category_4          object
vintage                 object
warranty                object
weight                 float64
wifi                    object
year                   float64
dtype: o

In [39]:
# convert float to integers
to_int = []
for col in to_int:
    df[col] = df[col].astype('int64')
    
# convert boolean to objects
bools = []
for var in bools:
    df[var] = df[var].astype('object')
    
# convert string to dates
dates = []
for var in dates:
    df[var] = pd.to_datetime(df[var].astype('str'), infer_datetime_format = True)

# 5. WORKING WITH FEATURES

### EXTRACT FEATURES FROM TEXT

In [40]:
# compute features
text_vars = ['product_description', 'product_name', 'store_name', 'category', 'material',
             'sub_category_1', 'sub_category_2', 'sub_category_3', 'sub_category_4']
print(df.shape)
df = add_text_features(df, strings = text_vars, k = 5, keep = True)
print(df.shape)

(11840, 32)
(11840, 95)


### DROP IRRELEVANT FEATURES

In [41]:
# remove columns with a single value
print(df.shape)
df = df.loc[:, df.nunique(dropna = False) != 1]
print(df.shape)

(11840, 95)
(11840, 95)


In [42]:
# remove irrelevant columns
print(df.shape)
drops = ['image_url']
for var in drops:
    del df[var]
print(df.shape)

(11840, 95)
(11840, 94)


### CREATE NEW FEATURES

In [43]:
df['image_pixels'] = df['image_height'] * df['image_width']

In [44]:
df['product_volume'] = df['product_height'] * df['product_length'] * df['product_width']

### ENCODE FACTORS

In [45]:
# convert categorical features
df = encode_factors(df, method = 'label')
df.head()

,age,author,brand,category,color,condition,duration,editor,id,image_height,image_width,images_count,material,price,product_description,product_height,product_length,product_name,product_size,product_width,shoe_size,store_name,sub_category_1,sub_category_2,sub_category_3,sub_category_4,vintage,warranty,weight,wifi,year,product_description_word_count,product_description_char_count,product_description_tfidf_0,product_description_tfidf_1,product_description_tfidf_2,product_description_tfidf_3,product_description_tfidf_4,product_name_word_count,product_name_char_count,product_name_tfidf_0,product_name_tfidf_1,product_name_tfidf_2,product_name_tfidf_3,product_name_tfidf_4,store_name_word_count,store_name_char_count,store_name_tfidf_0,store_name_tfidf_1,store_name_tfidf_2,store_name_tfidf_3,store_name_tfidf_4,category_word_count,category_char_count,category_tfidf_0,category_tfidf_1,category_tfidf_2,category_tfidf_3,category_tfidf_4,material_word_count,material_char_count,material_tfidf_0,material_tfidf_1,material_tfidf_2,material_tfidf_3,material_tfidf_4,sub_category_1_word_count,sub_category_1_char_count,sub_category_1_tfidf_0,sub_category_1_tfidf_1,sub_category_1_tfidf_2,sub_category_1_tfidf_3,sub_category_1_tfidf_4,sub_category_2_word_count,sub_category_2_char_count,sub_category_2_tfidf_0,sub_category_2_tfidf_1,sub_category_2_tfidf_2,sub_category_2_tfidf_3,sub_category_2_tfidf_4,sub_category_3_word_count,sub_category_3_char_count,sub_category_3_tfidf_0,sub_category_3_tfidf_1,sub_category_3_tfidf_2,sub_category_3_tfidf_3,sub_category_3_tfidf_4,sub_category_4_word_count,sub_category_4_char_count,sub_category_4_tfidf_0,sub_category_4_tfidf_1,sub_category_4_tfidf_2,sub_category_4_tfidf_3,sub_category_4_tfidf_4,image_pixels,product_volume
0,-1,-1,-1,0,0,0,2.0,-1,0,2552.0,3458.0,3,0,4.5,0,NaN,NaN,0,0,NaN,NaN,0,0,0,0,0,0,-1,200.0,-1,NaN,36,187,NaN,NaN,1.0,NaN,NaN,3,23,NaN,NaN,NaN,NaN,NaN,3,21,NaN,1.000000,NaN,NaN,NaN,1,4,NaN,NaN,NaN,NaN,1.0,3,14,0.726181,NaN,NaN,NaN,0.687503,4,29,NaN,NaN,NaN,1.0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,8824816.0,NaN
1,-1,-1,0,1,1,1,1.0,-1,1,2254.0,2486.0,2,1,15.0,1,NaN,NaN,1,-1,NaN,NaN,1,1,0,0,0,1,-1,1000.0,-1,NaN,8,41,NaN,NaN,NaN,NaN,NaN,3,17,NaN,NaN,NaN,NaN,NaN,5,43,NaN,1.000000,NaN,NaN,NaN,3,14,0.707107,NaN,NaN,0.707107,NaN,1,9,NaN,NaN,NaN,NaN,NaN,4,23,NaN,0.62645,0.531965,NaN,0.569714,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,5603444.0,NaN
2,-1,-1,1,2,2,2,1.0,-1,2,1536.0,1536.0,3,2,16.0,2,NaN,NaN,2,1,NaN,NaN,2,2,1,0,0,0,-1,360.0,-1,NaN,29,170,NaN,NaN,1.0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,3,21,NaN,0.442115,NaN,0.896958,NaN,2,15,NaN,1.0,NaN,NaN,NaN,3,21,NaN,0.508474,NaN,0.676422,0.532830,1,4,NaN,NaN,NaN,NaN,NaN,2,10,NaN,0.695038,NaN,0.718973,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,2359296.0,NaN
3,-1,-1,-1,1,-1,0,1.0,-1,3,1100.0,1100.0,2,3,24.0,3,NaN,NaN,3,-1,NaN,NaN,3,3,0,0,0,-1,-1,520.0,-1,NaN,38,216,0.604499,NaN,NaN,0.796606,NaN,4,24,1.0,NaN,NaN,NaN,NaN,2,11,NaN,1.000000,NaN,NaN,NaN,3,14,0.707107,NaN,NaN,0.707107,NaN,0,0,NaN,NaN,NaN,NaN,NaN,2,13,NaN,NaN,NaN,NaN,1.000000,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1210000.0,NaN
4,-1,-1,-1,3,3,3,0.0,-1,4,450.0,450.0,2,3,139.0,4,NaN,NaN,4,-1,NaN,NaN,4,4,0,0,0,-1,0,300.0,0,NaN,103,566,0.426706,0.451134,NaN,0.562311,0.546083,10,46,NaN,NaN,NaN,NaN,NaN,6,34,NaN,NaN,1.0,NaN,NaN,1,10,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,11,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,202500.0,NaN


# 6. EXPORT

In [46]:
# export CSV
df.to_csv('../data/data_v2.csv', index = False)
df.shape

(11840, 96)